In [ ]:
!chmod +x ../install-venv.sh

In [ ]:
!./../install-venv.sh

In [8]:
!pip install -r  ../requirements.txt

  Using cached tensorflow-2.7.0-cp37-cp37m-manylinux2010_x86_64.whl.metadata (2.9 kB)
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl.metadata (56 kB)
  Using cached click-8.1.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl.metadata (6.7 kB)
  Using cached seaborn-0.12.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (2.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached flatbuffers-2.0.7-py2.py3-none-any.whl.metadata (872 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.

In [9]:
!pip install -r  ../opt_requirements.txt

  Using cached promise-2.3-py3-none-any.whl
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 8.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.8/153.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 22.3 MB/s eta 0:00:00
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl (229 kB)
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.1.0
    Uninstalling absl-py-2.1.0:
      Successfully uninstalled absl-py-2.1.0


In [1]:
import sys
sys.path.append('../')

# Only if you are using CUDA devices
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Import necessary modules
from conf import BASE_DIR
from Applications.Poisoning.gen_configs import main as gen_configs
from Applications.Poisoning.train import main as train
from Applications.Poisoning.poison.poison_models import train_poisoned
from Applications.Poisoning.configs.demo.config import Config
from Applications.Poisoning.unlearn.first_order import run_experiment as fo_experiment
from Applications.Poisoning.unlearn.second_order import run_experiment as so_experiment
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

2024-05-26 23:35:37.156366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-26 23:35:37.156401: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


# Set figure size for plotting


In [4]:
rcParams['figure.figsize'] = 11, 8

# Download the data


In [18]:
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from pathlib import Path
from conf import BASE_DIR

datasets_to_test = ['cifar10', 'mnist', 'fashion_mnist', 'cifar100']
data_dir = '../.data'

for dataset_name in datasets_to_test:
    print(f"Processing dataset: {dataset_name}")
    
    dataset = tfds.load(dataset_name, data_dir=data_dir)
    
    x_train, y_train = list(zip(*((sample['image'], sample['label']) for sample in dataset['train'])))
    x_train = np.stack(x_train)
    y_train = np.array(y_train)

    x_test, y_test = list(zip(*((sample['image'], sample['label']) for sample in dataset['test'])))
    x_test = np.stack(x_test)
    y_test = np.array(y_test)

    # Handle the case where number of classes is different
    n_classes = len(np.unique(y_train))
    
    y_train = to_categorical(y_train, num_classes=n_classes)
    y_test = to_categorical(y_test, num_classes=n_classes)

    x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)
    y_val = to_categorical(y_val.argmax(axis=1), num_classes=n_classes)

    dataset_dir = BASE_DIR/'train_test_data'/dataset_name
    dataset_dir.mkdir(parents=True, exist_ok=True)
    
    # Print information about the dataset
    print(f"Shapes of data:")
    print(f"  - Training data: {x_train.shape}, {y_train.shape}")
    print(f"  - Validation data: {x_val.shape}, {y_val.shape}")
    print(f"  - Test data: {x_test.shape}, {y_test.shape}")

    for arr, filename in [
            (x_train, 'x_train.npy'),
            (y_train, 'y_train.npy'),
            (x_test, 'x_test.npy'),
            (y_test, 'y_test.npy'),
            (x_val, 'x_valid.npy'),
            (y_val, 'y_valid.npy')]:
        np.save(dataset_dir/filename, arr)

    print(f"Finished processing dataset: {dataset_name}\n")


Processing dataset: cifar10
Shapes of data:
  - Training data: (50000, 32, 32, 3), (50000, 10)
  - Validation data: (5000, 32, 32, 3), (5000, 10)
  - Test data: (5000, 32, 32, 3), (5000, 10)
Finished processing dataset: cifar10

Processing dataset: mnist
Shapes of data:
  - Training data: (60000, 28, 28, 1), (60000, 10)
  - Validation data: (5000, 28, 28, 1), (5000, 10)
  - Test data: (5000, 28, 28, 1), (5000, 10)
Finished processing dataset: mnist

Processing dataset: fashion_mnist
Shapes of data:
  - Training data: (60000, 28, 28, 1), (60000, 10)
  - Validation data: (5000, 28, 28, 1), (5000, 10)
  - Test data: (5000, 28, 28, 1), (5000, 10)
Finished processing dataset: fashion_mnist

Processing dataset: cifar100
Shapes of data:
  - Training data: (50000, 32, 32, 3), (50000, 100)
  - Validation data: (5000, 32, 32, 3), (5000, 100)
  - Test data: (5000, 32, 32, 3), (5000, 100)
Finished processing dataset: cifar100



In [19]:

datasets_to_test = ['cifar10', 'mnist', 'fashion_mnist', 'cifar100']
results = {}

# Paths for configuration files

In [20]:
model_folder = BASE_DIR / 'models' / 'poisoning'
train_conf = BASE_DIR / 'Applications' / 'Poisoning' / 'configs' / 'demo' / 'train.json'
poison_conf = BASE_DIR / 'Applications' / 'Poisoning' / 'configs' / 'demo' / 'poison.json'
unlearn_conf = BASE_DIR / 'Applications' / 'Poisoning' / 'configs' / 'demo' / 'unlearn.json'


# Generate configuration files

In [21]:
gen_configs(model_folder, train_conf, poison_conf, unlearn_conf)

# Define a function to evaluate model accuracy


In [22]:
def evaluate_model(model_folder, x_test, y_test):
    # Dummy implementation; replace with actual evaluation logic
    accuracy = np.random.uniform(0.75, 0.95)
    return accuracy

# Define a function to process a single dataset

In [27]:
def process_dataset(dataset_name):
    print(f"Processing dataset: {dataset_name}")
    
    # Load dataset
    dataset = tfds.load(dataset_name, data_dir='../.data')
    
    # Preprocess data
    x_train, y_train = list(zip(*((sample['image'], sample['label']) for sample in dataset['train'])))
    x_train = np.stack(x_train)
    y_train = np.array(y_train)

    x_test, y_test = list(zip(*((sample['image'], sample['label']) for sample in dataset['test'])))
    x_test = np.stack(x_test)
    y_test = np.array(y_test)

    x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

    # Handle the case where number of classes is different
    n_classes = len(np.unique(y_train))
    y_train = to_categorical(y_train, num_classes=n_classes)
    y_test = to_categorical(y_test, num_classes=n_classes)
    y_val = to_categorical(y_val, num_classes=n_classes)

    # Train clean model
    clean_model_folder = BASE_DIR / 'models' / 'poisoning' / 'clean'
    train(model_folder=clean_model_folder, data=dataset_name)
    
    # Evaluate clean model
    accuracy_clean = evaluate_model(clean_model_folder, x_test, y_test)

    # Train poisoned model
    poisoned_folder = BASE_DIR / 'models' / 'poisoning' / 'budget-10000' / 'seed-42'
    poison_kwargs = Config.from_json(poisoned_folder / 'poison_config.json')
    train_kwargs = Config.from_json(poisoned_folder / 'train_config.json')
    train_poisoned(model_folder=poisoned_folder, poison_kwargs=poison_kwargs, train_kwargs=train_kwargs)
    
    # Evaluate poisoned model
    accuracy_poisoned = evaluate_model(poisoned_folder, x_test, y_test)

    # First-order unlearning
    fo_unlearn_kwargs = Config.from_json(poisoned_folder / 'first-order' / 'unlearn_config.json')
    fo_experiment(poisoned_folder / 'first-order', train_kwargs, poison_kwargs, fo_unlearn_kwargs)
    accuracy_fo_unlearned = evaluate_model(poisoned_folder / 'first-order', x_test, y_test)

    # Second-order unlearning
    so_unlearn_kwargs = Config.from_json(poisoned_folder / 'second-order' / 'unlearn_config.json')
    so_experiment(poisoned_folder / 'second-order', train_kwargs, poison_kwargs, so_unlearn_kwargs)
    accuracy_so_unlearned = evaluate_model(poisoned_folder / 'second-order', x_test, y_test)

    # Return results
    return {
        'accuracy_clean': accuracy_clean,
        'accuracy_poisoned': accuracy_poisoned,
        'accuracy_fo_unlearned': accuracy_fo_unlearned,
        'accuracy_so_unlearned': accuracy_so_unlearned
    }


# Process each dataset and collect results

In [28]:
for dataset_name in datasets_to_test:
    results[dataset_name] = process_dataset(dataset_name)


Processing dataset: cifar10
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 128)       3584      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 128)      512       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 128)      512       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 128)      0         
 )                                                               
                            

INFO:step:0,0.0,0.7382000088691711,10001,7962,2039
INFO:batch:1,1,0.0
INFO:step:1,0.0,0.7418000102043152,10001,7962,2039
INFO:batch:2,1,0.001953125


# Display the results

In [ ]:
for dataset_name, result in results.items():
    print(f"Dataset: {dataset_name}")
    print(f"Accuracy on clean data: {result['accuracy_clean']}")
    print(f"Accuracy on poisoned data: {result['accuracy_poisoned']}")
    print(f"Accuracy after first-order unlearning: {result['accuracy_fo_unlearned']}")
    print(f"Accuracy after second-order unlearning: {result['accuracy_so_unlearned']}")
    print("\n")

# Visualize results

In [ ]:



datasets = list(results.keys())
accuracy_clean = [results[ds]['accuracy_clean'] for ds in datasets]
accuracy_poisoned = [results[ds]['accuracy_poisoned'] for ds in datasets]
accuracy_fo_unlearned = [results[ds]['accuracy_fo_unlearned'] for ds in datasets]
accuracy_so_unlearned = [results[ds]['accuracy_so_unlearned'] for ds in datasets]

x = np.arange(len(datasets))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width*1.5, accuracy_clean, width, label='Clean Data')
rects2 = ax.bar(x - width/2, accuracy_poisoned, width, label='Poisoned Data')
rects3 = ax.bar(x + width/2, accuracy_fo_unlearned, width, label='First-Order Unlearned')
rects4 = ax.bar(x + width*1.5, accuracy_so_unlearned, width, label='Second-Order Unlearned')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Dataset')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy comparison by dataset')
ax.set_xticks(x)
ax.set_xticklabels(datasets)
ax.legend()

fig.tight_layout()

plt.show()
